In [ ]:
# import drive files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
using_colab = True
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything-2.git'

In [ ]:
#Mexico TEST
 image_folder = '/content/drive/MyDrive/mexico_dataset/labels'  # path of all the test images
 gt_folder = '/content/drive/MyDrive/mexico_dataset/masks'  # path of ground truth masks
 result_folder = '/content/drive/MyDrive/trabalho/SAM2/BUS_all_dataset_resize/test/results' # path of evaluation result images
 sam2_checkpoint = "/content/drive/MyDrive/sam2_hiera_large.pt"  #checkpoints (coefficients) of the SAM2 model

 model_cfg = "sam2_hiera_l.yaml"

In [ ]:
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import random
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

# select the device for computation
if torch.cuda.is_available():
    device = torch.device("cuda")
    current_device = torch.cuda.current_device()
    print("Current GPU device:", torch.cuda.get_device_name(current_device))
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

if device.type == "cuda":
    # use bfloat16 for the entire notebook
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
elif device.type == "mps":
    print(
        "\nSupport for MPS devices is preliminary. SAM 2 is trained with CUDA and might "
        "give numerically different outputs and sometimes degraded performance on MPS. "
        "See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion."
    )

# Define functions
np.random.seed(3)

def show_anns(anns, borders=True):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:, :, 3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.5]])
        img[m] = color_mask
        if borders:
            import cv2
            contours, _ = cv2.findContours(m.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            contours = [cv2.approxPolyDP(contour, epsilon=0.01, closed=True) for contour in contours]
            cv2.drawContours(img, contours, -1, (0, 0, 1, 0.4), thickness=1)
    ax.imshow(img)

def show_mask(mask, ax, random_color=False, borders=True):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask = mask.astype(np.uint8)
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    if borders:
        import cv2
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        contours = [cv2.approxPolyDP(contour, epsilon=0.01, closed=True) for contour in contours]
        mask_image = cv2.drawContours(mask_image, contours, -1, (1, 1, 1, 0.5), thickness=2)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], coords[labels==0][:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0, 0, 0, 0), lw=2))

def show_masks(image, masks, scores, point_coords=None, box_coords=None, input_labels=None, borders=True):
    for i, (mask, score) in enumerate(zip(masks, scores)):
        plt.figure(figsize=(5, 5))
        plt.imshow(image)
        show_mask(mask, plt.gca(), borders=borders)
        if point_coords is not None:
            assert input_labels is not None
            show_points(point_coords, input_labels, plt.gca())
        if box_coords is not None:
            show_box(box_coords, plt.gca())
        if len(scores) > 1:
            plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
        plt.axis('off')
        plt.show()

def threshold_mask(probability_mask, threshold=0.5):
    return (probability_mask >= threshold).astype(int)

def dice_coefficient(pred_mask, true_mask):
    intersection = np.sum(pred_mask * true_mask)
    return (2. * intersection) / (np.sum(pred_mask) + np.sum(true_mask))

def calculate_iou(binary_mask1, binary_mask2):
    intersection = np.logical_and(binary_mask1, binary_mask2).sum()
    union = np.logical_or(binary_mask1, binary_mask2).sum()
    return 1.0 if union == 0 else intersection / union

def find_median_point(gt):
    """
    Finds the median point of all white pixels (foreground region of the mask).

    gt: binary mask (numpy array)

    Returns:
    median_x, median_y: coordinates of the median point
    """
    white_pixels = np.column_stack(np.where(gt == 1))

    if len(white_pixels) > 0:
        median_x = np.median(white_pixels[:, 0]).astype(int)
        median_y = np.median(white_pixels[:, 1]).astype(int)

        if gt[median_x, median_y] == 1:
            return median_x, median_y
        else:
            print("The computed median is not in the white region. Selecting the closest point.")
            closest_idx = np.argmin(np.linalg.norm(white_pixels - np.array([median_x, median_y]), axis=1))
            closest_point = white_pixels[closest_idx]
            return closest_point[0], closest_point[1]
    else:
        raise ValueError("No white pixels found in the mask.")

def find_rect_box(gt):
    white_pixels = np.column_stack(np.where(gt == 1))
    if len(white_pixels) > 0:
        x_min, y_min = np.min(white_pixels, axis=0)
        x_max, y_max = np.max(white_pixels, axis=0)
    else:
        print("Invalid mask")
    input_box = np.array([y_min, x_min, y_max, x_max])
    return input_box

def segment(prompt_type, prompt, image_path, predictor=None, show_mask=False):
    if predictor is None:
        raise ValueError("Predictor must be initialized before calling segment function")

    image = Image.open(image_path)
    image = np.array(image.convert("RGB"))
    predictor.set_image(image)

    if prompt_type == 1:  # single point
        input_label = np.array([1])
        masks, scores, _ = predictor.predict(
            point_coords=prompt,
            point_labels=input_label,
            multimask_output=False,
        )
        pred_mask = threshold_mask(masks)
        if show_mask:
            show_masks(image, masks, scores, point_coords=input_point, input_labels=input_label, borders=True)
    elif prompt_type == 2:  # single box
        masks, scores, _ = predictor.predict(
            point_coords=None,
            point_labels=None,
            box=prompt[None, :],
            multimask_output=False,
        )
        pred_mask = threshold_mask(masks)
        if show_mask:
            show_masks(image, masks, scores, box_coords=input_box)
    return pred_mask


In [ ]:
sam2_model = build_sam2(model_cfg, sam2_checkpoint, device=device)
predictor = SAM2ImagePredictor(sam2_model)

SAM Point

In [ ]:
import os
import numpy as np
from PIL import Image

# Initialize arrays for storing the results
dice_arr = []
iou_arr = []
image_count = 0

# Loop through all images in the image folder
for filename in os.listdir(image_folder):
    if filename.endswith(".png") or filename.endswith(".jpg"):  # Only operate on image files
        image_path = os.path.join(image_folder, filename)

        # Get the base name of the image (without the extension)
        image_base_name = os.path.splitext(filename)[0]

        # List all masks that start with the same base name
        masks = [mask for mask in os.listdir(gt_folder) if mask.startswith(image_base_name)]

        # If no masks are found, skip the image
        if not masks:
            print(f"No mask found for {filename}. Skipping...")
            continue

        # Process each mask associated with this image
        for mask_name in masks:
            gt_path = os.path.join(gt_folder, mask_name)
            gt = Image.open(gt_path)
            gt = np.array(gt.convert("L")) / 255  # Convert to binary (0 or 1)

            # Generate prompt
            median_x, median_y = find_median_point(gt)
            input_point = np.array([[median_y, median_x]])  # Note that it's (y, x)
            input_label = np.array([1])

            # Get the predicted mask using the segment function
            pred_mask = segment(1, input_point, image_path, predictor, show_mask=True)  # 1 stands for Single Point Prompt Type

            # Dice Coefficient
            dice = dice_coefficient(pred_mask, gt)
            print(f"Dice Coefficient for {filename} (mask {mask_name}): {dice:.4f}")
            dice_arr.append(dice)

            # IoU (Intersection over Union)
            intersection = np.sum((pred_mask == 1) & (gt == 1))
            union = np.sum((pred_mask == 1) | (gt == 1))
            iou = intersection / union if union > 0 else 0
            print(f"IoU for {filename} (mask {mask_name}): {iou:.4f}")
            iou_arr.append(iou)

        # Increment the image count after processing all masks for this image
        image_count += 1

        # If we have processed 10 images, stop further processing
        #if image_count >= 2:
            #break

    # If we've processed 10 images, exit the outer loop
    #if image_count >= 2:
        #break

# Calculate and print the averages directly
if dice_arr:
    print(f"Mean Dice Coefficient: {np.mean(dice_arr):.4f}")
    print("All Dice Coefficients:", dice_arr)

if iou_arr:
    print(f"Mean IoU: {np.mean(iou_arr):.4f}")
    print("All IoU values:", iou_arr)